In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
from tensorflow.keras.applications import Xception

print(tf.__version__)
print(tf.keras.__version__)

# TTA Prediction

In [ ]:
# make a prediction using test-time augmentation

def tta_prediction(datagen, model, image, aug_img_number):
    # convert image into dataset
    samples = np.expand_dims(image, 0)
    
    print(samples.shape)
    # prepare iterator
    it = datagen.flow(samples, batch_size=aug_img_number)
    
    # make predictions for each augmented image
    preds = model.predict_generator(it, steps=aug_img_number, verbose=0)
    print(preds)
    # sum across predictions
    # do either sum or mean whatever you like
    summed = np.sum(preds, axis=0)
    
    # argmax across classes
    return np.argmax(summed)

In [ ]:
train_images_path = '../input/panda-cancer-l2-sqtile32-v1/train_squaretile_images/'
train_df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
def get_image():
    
    image = plt.imread(train_images_path + train_df.loc[2]['image_id'] + '.png')
    
    return image

In [ ]:
train_df.loc[2]

In [ ]:
from tensorflow.keras.models import load_model
# select type of augmentation to do 
tta_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip = True,
        vertical_flip = True,
        rotation_range=90 
)


# get prediction as following for one image
model2 = load_model('../input/pandaxceptiononlessdata/xception-16-0.59.hdf5', compile=False) 


In [ ]:
image =  get_image()
plt.imshow(get_image())

In [ ]:
%%time

aug_img_number = 3 # numer of images to generate 
pred = tta_prediction(tta_datagen, model2, image, aug_img_number)

In [ ]:
print(pred)

# USING Albumentation

In [ ]:
import albumentations
augT  = albumentations.Transpose(p=1)
augVF = albumentations.VerticalFlip(p=1)
augHF = albumentations.HorizontalFlip(p=1)

def tta_pred(image):
    image = image.reshape((1, 768, 768, 3))
    
    image1 = augHF(image = image)['image']
    pred1 = model2.predict(image1, batch_size = 1 )
    
    image2 = augHF(image = image)['image']
    pred2 = model2.predict(image2, batch_size = 1)
    
    image3 = augHF(image = image)['image']
    pred3 = model2.predict(image3, batch_size = 1)
    
    final_pred = ( pred1 + pred2 + pred3 ) / 3

    return final_pred


In [ ]:
%%time
image =  get_image()

In [ ]:
%%time
#for i in range(2):
print(tta_pred(image))

In [ ]:
plt.imshow(image)

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    if train_df['image_path'][i] in [path_list_1, path_list_2]:
        new_img_df = new_img_df.append(train_df.loc[i])

In [ ]:
train_df_not_01 = train_df[(train_df['isup_grade'] != 0) & (train_df['isup_grade'] != 1)]
train_class0 = train_df[train_df['isup_grade'] == 0].sample(frac=0.489)
train_class1 = train_df[train_df['isup_grade'] == 1].sample(frac=0.498)

print(train_df_not_01['isup_grade'].value_counts())
print(train_class0['isup_grade'].value_counts())
print(train_class1['isup_grade'].value_counts())

In [ ]:
final_train = train_df_not_01
final_train = final_train.append(train_class0)
final_train = final_train.append(train_class1)
final_train.reset_index(inplace = True) 

print(final_train['isup_grade'].value_counts())
print(final_train.head())

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold


def getFold(kfold):
    
    split_size = kfold
    skFold = StratifiedKFold(n_splits = split_size, random_state=42, shuffle = True)
    
    data_list = []

    for train_index, test_index in skFold.split(train_df["image_id"], train_df["isup_grade"] ):
        
        X_train = train_df[["image_id"]].iloc[train_index]
        X_test  = train_df[["image_id"]].iloc[test_index]
    
        y_train = train_df[["isup_grade"]].iloc[train_index]
        y_test  = train_df[["isup_grade"]].iloc[test_index]
       
        df_train = pd.DataFrame({"image_id":X_train['image_id'], "isup_grade":y_train['isup_grade']}, dtype=str)
        df_valid = pd.DataFrame({"image_id":X_test['image_id'], "isup_grade":y_test['isup_grade']}, dtype=str)
        
        dataDict = {'train':df_train,'valid':df_valid}
        
        data_list.append(dataDict)
        
    return data_list    


In [ ]:
data_fold = getFold(2)

In [ ]:
for data in data_fold[:1]:
    
    fold1 = data.get('train')
    fold2 = data.get('valid')
    
    print(fold1['isup_grade'].value_counts())
    print(fold2['isup_grade'].value_counts())

    print()
    print('------------------------------------------------------')
    print()
    print('======================================================')
    print('======================================================')
    print('======================================================')
    print('======================================================')    